In [1]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob

In [2]:
G48_raw_data = sorted(glob('./raw_data/G48/G*.txt'))

In [3]:
G48_raw_data_name = list(map(lambda x:x[15:],G48_raw_data))

In [5]:
i = 0
all_data = pd.DataFrame()
for file in G48_raw_data:
    data = pd.read_csv(file, sep = '\t')
    data['data_name'] = G48_raw_data_name[i]
    data.columns = data.columns.str.replace('  ','')
    data.columns = data.columns.str.replace('G48 -','G48-') #有些奇怪的間隔
    data['data_point'] = data.index
    all_data = pd.concat([data,all_data])
    all_data = all_data.drop([0]) #消除單位Deg.
    all_data.reset_index(drop = True)
    i = i + 1

C:\Users\AutoUser\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
all_data.columns

Index(['G48-PTC1', 'G48-PTC10', 'G48-PTC11', 'G48-PTC12', 'G48-PTC13',
       'G48-PTC14', 'G48-PTC2', 'G48-PTC25', 'G48-PTC26', 'G48-PTC27',
       'G48-PTC28', 'G48-PTC29', 'G48-PTC3', 'G48-PTC31', 'G48-PTC32',
       'G48-PTC33', 'G48-PTC34', 'G48-PTC35', 'G48-PTC36', 'G48-PTC37',
       'G48-PTC4', 'G48-PTC5', 'G48-PTC6', 'G48-PTC7', 'G48-PTC8', 'G48-PTC9',
       'Unnamed: 4', 'Unnamed: 8', 'data_name', 'data_point'],
      dtype='object')

In [7]:
#drop 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8' 的columns
#rows出現 Deg的資料 欄位
for column in all_data.columns:
    if column[0] == 'U':
        all_data.drop([column],axis = 1,inplace = True)

In [8]:
all_data['data_name'] = all_data['data_name'].apply(lambda x:x[:-11]) #去掉 _Export.txt

In [9]:
all_data['data_name'].tail()

376    G48-AC(12X30)20160413-003
377    G48-AC(12X30)20160413-003
378    G48-AC(12X30)20160413-003
379    G48-AC(12X30)20160413-003
380    G48-AC(12X30)20160413-003
Name: data_name, dtype: object

In [10]:
all_data['recipes'] = all_data['data_name'].str.extract(r'(G\d+)',expand = False) #抓出所屬的Recipes
all_data['autoclave'] = all_data['data_name'].str.extract(r'(AC\(\d+x\d+\))',expand = False,flags=re.IGNORECASE) #用正則抓出 所屬的熱壓爐
all_data['date'] = all_data['data_name'].str.extract(r'(\d{7,8})',expand = False) #抓出日期
all_data['number'] = all_data['data_name'].str.extract(r'(00\d{1,3})',expand = False)#抓出所屬編號
all_data.drop(['data_name'],axis = 1,inplace = True)

In [11]:
all_data.columns = all_data.columns.str.replace('G48-','')

In [12]:
PTC_list = []
for column in all_data.columns:
    if column[0] == 'P' and column[-1] != 't':
        PTC_list.append(column)

In [13]:
all_data['PTC_count'] = len(PTC_list) - (all_data.shape[1] - all_data.count(axis = 1)) 
#每筆資料所收集溫度參數的個數

In [14]:
all_data.columns

Index(['PTC1', 'PTC10', 'PTC11', 'PTC12', 'PTC13', 'PTC14', 'PTC2', 'PTC25',
       'PTC26', 'PTC27', 'PTC28', 'PTC29', 'PTC3', 'PTC31', 'PTC32', 'PTC33',
       'PTC34', 'PTC35', 'PTC36', 'PTC37', 'PTC4', 'PTC5', 'PTC6', 'PTC7',
       'PTC8', 'PTC9', 'data_point', 'recipes', 'autoclave', 'date', 'number',
       'PTC_count'],
      dtype='object')

In [15]:
G48_data = pd.DataFrame()

In [16]:
for PTC in PTC_list:
    data = all_data.loc[~all_data[PTC].isnull(),
             ['number', 'date', 'autoclave', 'recipes'
              ,'TemParameter', 'PTC_count','data_point',PTC]] #將各別 PTC 非0的數據挑出
    data['TemParameter'] = PTC  #建立 TemParameter 對應所屬來的PTC
    data['value'] = data[PTC]  # 賦予value值 統一
    data.drop([PTC],inplace=True,axis = 1)
    G48_data = pd.concat([G48_data,data])

C:\Users\AutoUser\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [17]:
G48_data.reset_index(drop=True, inplace=True)

In [18]:
G48_data['TemParameter'].value_counts()

PTC28    6642
PTC26    6224
PTC32    5838
PTC3     5504
PTC8     5212
PTC2     4883
PTC11    4869
PTC34    4821
PTC27    4418
PTC5     4382
PTC33    3779
PTC4     3676
PTC12    3457
PTC9     3390
PTC29    3095
PTC36    2788
PTC31    2715
PTC10    2480
PTC35    1423
PTC6     1405
PTC7      766
PTC25     765
PTC1      759
PTC37     379
PTC14     341
PTC13     341
Name: TemParameter, dtype: int64

In [19]:
G48_data.to_csv('./training_data/G48_data.csv')